In [ ]:
import geopandas as gpd
import pandas as pd
from gerrychain import Graph
import random
import networkx as nx
import numpy as np

import facefinder
import numpy as np
import pandas as pd
import random
import pickle
import csv
import copy
import statistics
import math
import gerrychain
import networkx
import matplotlib as mpl
import matplotlib.pyplot as plt
import networkx as nx
import os
import json
import argparse
from functools import partial
from gerrychain.tree import bipartition_tree as bpt
from gerrychain import Graph, MarkovChain
from gerrychain import accept
from gerrychain.constraints import (Validator, single_flip_contiguous,
                                    within_percent_of_ideal_population, UpperBound)
from gerrychain.proposals import propose_random_flip, propose_chunk_flip
from gerrychain.accept import always_accept
from gerrychain.updaters import Election, Tally, cut_edges
from gerrychain import GeographicPartition
from gerrychain.partition import Partition
from gerrychain.proposals import recom
from gerrychain.metrics import mean_median, efficiency_gap
from gerrychain.tree import recursive_tree_part, bipartition_tree_random, PopulatedGraph, random_spanning_tree
from collections import defaultdict
from datetime import datetime
import time
from concurrent.futures import ProcessPoolExecutor
import logging

vtd = gpd.read_file("./data/NC_VTD.shp").to_crs("EPSG:4269")
bg = gpd.read_file("./data/tl_2016_37_bg.shp").to_crs("EPSG:4269")
blocks = gpd.read_file("./data/blocks/tl_2016_37_tabblock10.shp").to_crs("EPSG:4269")

In [ ]:
#joined_within = gpd.sjoin(blocks, vtd, how = 'left', predicate = 'within' )
#joined_within[["GEOID10", "VTD"]].groupby("GEOID10").agg('count').value_counts()

i = 0 # only change to 1 to experiment with block groups;
geo_table = [ blocks, bg][i]
geo_id = ["GEOID10", "GEOID"][i]

joined = gpd.sjoin(geo_table, vtd, how = 'left',  predicate = 'intersects' )
mapping = joined[[geo_id, "VTD"]]

In [ ]:
display(joined[[geo_id, "VTD"]].groupby(geo_id).agg('count').value_counts())
#counts = joined[[geo_id, "VTD"]].groupby(geo_id).agg('count').rename(columns = {"VTD" : "NUM INTERSECTIONS"})
#m = joined.merge(counts, left_on = geo_id, right_index = True)
#m[m["NUM INTERSECTIONS"] <= 5].plot("NUM INTERSECTIONS", legend = True)

In [ ]:
# Intersect geos with VTDs, compute area ratios
geo_table_with_map = geo_table.merge(mapping)#.merge(vtd[["VTD", "geometry"]], suffixes = ('_block', '_vt') )
geo_with_vtd = geo_table_with_map.merge(vtd,
                                                      left_on = "VTD", right_on = "VTD",
                                                      suffixes = ('_geo', '_vtd'))

geo_geos = gpd.GeoSeries(geo_with_vtd.geometry_geo).to_crs("ESRI:103500")
geo_with_vtd['geo_intersect'] = geo_geos.intersection(gpd.GeoSeries(geo_with_vtd.geometry_vtd).to_crs("ESRI:103500"))
geo_with_vtd['geo_area'] = geo_geos.area
geo_with_vtd['intersect_area'] = gpd.GeoSeries(geo_with_vtd.geo_intersect).to_crs("ESRI:103500").area
geo_with_vtd['area_proportion'] =  geo_with_vtd['intersect_area'] / geo_with_vtd['geo_area']

In [ ]:
pops = pd.read_csv('./data/us2016_block_pops.csv')
pops['block_fips'] = pops.block_fips.astype("str")
vote_fields = ["EL16G_GV_R" , "EL16G_GV_D", "TOTPOP"]
fields = ['VTD', "GEOID10", 'geometry', 'area_proportion' ] +  vote_fields
geos_simple = geo_with_vtd.rename(columns = {"geometry_geo" : "geometry" })[fields]
geo_with_pop = geos_simple.merge(pops, left_on = geo_id, right_on = 'block_fips',)

In [ ]:
# VTD level sums -- doing because the VTD tot pop is generally not equal to the sum of the block pops in it
# This indicates some data level issue , probably in addition to the non exact containment issue.
# Code to check:
# merged = geo_with_pop[["TOTPOP", "VTD"]].merge(geo_with_pop[["pop2016", "VTD"]].groupby("VTD").agg(sum).reset_index(), left_on = "VTD", right_on = "VTD")
# (merged.TOTPOP / merged.pop2016).sort_values().apply(lambda x : round(x,1) ).value_counts()
# But ignoring for now.
vtds_block_pops = geo_with_pop[["pop2016", "VTD"]].groupby("VTD").agg(sum)
vtds_block_pops.columns = ["pop2016_VTD_total"]
geo_with_pop = geo_with_pop.merge(vtds_block_pops, left_on = "VTD", right_index = True)
geo_with_pop['pop_proportion'] = geo_with_pop.pop2016 /  geo_with_pop.pop2016_VTD_total

# Filter out and set to zero for the ones where the intersectiokn is too small
# NB: I did not include this step the first itme running it; TODO -- rerun.


In [ ]:
vote_fields = ["EL16G_GV_R" , "EL16G_GV_D", "TOTPOP"]
for field in vote_fields:
    # Eventually replace with ecological inference or something
    geo_with_pop[field + "_block"] = geo_with_pop['pop_proportion'] * geo_with_pop[field]

In [ ]:
gdf_dedupped = geo_with_pop.sort_values('area_proportion').drop_duplicates('GEOID10', keep='last')
gdf_subset.pop2016.sum() / geo_with_pop.pop2016.sum()

In [ ]:
gdf_subset = gdf_dedupped[['VTD', "GEOID10", 'geometry', "TOTPOP", "pop2016", "pop_proportion", 'area_proportion' ] +  [ x + "_block" for x in vote_fields] ]
gdf_subset = gpd.GeoDataFrame(gdf_subset)

test = gpd.GeoDataFrame(gdf_subset, geometry="geometry", crs= "ESRI:103500")#.head(10)
test.to_file('./processed_data_outputs/blocks_with_votes.shp')

In [ ]:
test_in = gpd.read_file('./processed_data_outputs/blocks_with_votes.shp')

In [ ]:
gdf_subset['pos'] = gdf_subset.centroid
gdf_subset = gdf_subset.rename(columns = { "EL16G_GV_D_block" : "dem", "EL16G_GV_R_block" : "rep"})
gdf_subset['dem_prop'] = gdf_subset.dem / gdf_subset.TOTPOP_block
gdf_subset['rep_prop'] = gdf_subset.rep / gdf_subset.TOTPOP_block


In [ ]:
blocks_graph = Graph.from_geodataframe(gdf_subset, ignore_errors=True)#

In [ ]:
blocks_graph.to_json('./processed_data_outputs/blocks_graph.json', include_geometries_as_geojson=True)

In [ ]:
blocks_graph_in = Graph.from_json('./processed_data_outputs/blocks_graph.json')

In [ ]:
def condensce_by_partition(graph, part):
    # given graph, and label field for each node, return graph where all the nodes with the same lable have been contracted
    # also, add up the pops and votes in each block
    graph_copy = graph.copy()
    
    for supernode in part.parts.values():
        nodes = list(supernode)
        for node in nodes[1:]:
            
            tot_pop = graph_copy.nodes[nodes[0]]['TOTPOP_block'] + graph_copy.nodes[node]['TOTPOP_block']
            tot_dem = graph_copy.nodes[nodes[0]]['dem']+ graph_copy.nodes[node]['dem']
            tot_rep = graph_copy.nodes[nodes[0]]['rep'] + graph_copy.nodes[node]['rep']
            nx.contracted_nodes(graph_copy, nodes[0], node, copy = False)
            graph_copy.nodes[nodes[0]]['TOTPOP_block'] = tot_pop
            graph_copy.nodes[nodes[0]]['dem'] = tot_dem
            graph_copy.nodes[nodes[0]]['rep'] = tot_rep
            
    return graph_copy